In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageFilter

# تعريف الألوان
BG_COLOR = "#A163F7"
BTN_COLOR = "#6F88FC"
TEXT_COLOR = "#f8f8f2"
ACTIVE_BTN_COLOR = "#45E3FF"

# إعداد النافذة الرئيسية
root = tk.Tk()
root.title("تحويل الصور")
root.geometry("350x450")
root.configure(bg=BG_COLOR)

# تعريف المتغيرات العامة
selected_image = None
applied_filters = {}
image_window = None
image_label = None

# تحميل الصورة
def load_image():
    global selected_image, image_label_text
    file_path = filedialog.askopenfilename()
    try:
        selected_image = Image.open(file_path)
        display_image(selected_image)
        image_label_text.set("الصورة المحددة: " + file_path)
    except:
        messagebox.showerror("خطأ", "فشل تحميل الصورة")

# عرض الصورة
def display_image(image):
    global image_window, image_label, image_tk
    if image_window is None:
        image_window = tk.Toplevel(root)
        image_window.protocol("WM_DELETE_WINDOW", close_image_window)
        image_label = tk.Label(image_window)
        image_label.pack()
    image_tk = ImageTk.PhotoImage(image)
    image_label.config(image=image_tk)
    image_label.image = image_tk

# إغلاق النافذة التي تعرض الصورة
def close_image_window():
    global image_window
    if image_window is not None:
        image_window.destroy()
        image_window = None

# تطبيق الفلاتر على الصورة
def apply_filters():
    global selected_image, applied_filters
    if selected_image is not None:
        filtered_image = selected_image.copy()
        for filter_name, filter_data in applied_filters.items():
            if filter_data["enabled"].get():
                slider_value = None if "slider" not in filter_data else filter_data["slider"].get()
                filtered_image = filter_name(filtered_image, slider_value)
        display_image(filtered_image)

# الفلاتر المحددة
def bw_image(image, _):
    return image.convert('L')

def blur_image(image, radius):
    return image.filter(ImageFilter.GaussianBlur(radius))

def flip_image(image, _):
    return image.transpose(Image.FLIP_LEFT_RIGHT)

def resize_image(image, ratio):
    width, height = image.size
    new_width = int(width * ratio)
    new_height = int(height * ratio)
    return image.resize((new_width, new_height))

def binary_image(image, _):
    return image.convert('1')

# تحميل الشعار وإضافته إلى الواجهة
logo_image = Image.open("logoEVC.png")
new_width = 150
new_height = int(logo_image.height * (new_width / logo_image.width))
logo_image_resized = logo_image.resize((new_width, new_height), Image.ANTIALIAS)
logo_image_tk = ImageTk.PhotoImage(logo_image_resized)
logo_label = tk.Label(root, image=logo_image_tk, bg=BG_COLOR)
logo_label.pack(pady=10)

# إعداد الأزرار
load_button = tk.Button(root, text="تحميل الصورة", command=load_image, bg=BTN_COLOR, fg=TEXT_COLOR, activebackground=ACTIVE_BTN_COLOR)
apply_button = tk.Button(root, text="تطبيق التغييرات", command=apply_filters, bg=BTN_COLOR, fg=TEXT_COLOR, activebackground=ACTIVE_BTN_COLOR)
image_label_text = tk.StringVar()
image_label_text.set("الصورة لم يتم تحميلها بعد")
image_label_main = tk.Label(root, textvariable=image_label_text, bg=BG_COLOR, fg=TEXT_COLOR)

filters = {
    bw_image: "صورة أبيض وأسود",
    blur_image: "بلر الصورة",
    flip_image: "قلب الصورة",
    resize_image: "تغيير حجم الصورة",
    binary_image: "صورة ثنائية (0، 1)"
}

# إعداد الفلاتر
for filter_func, filter_text in filters.items():
    filter_frame = tk.Frame(root, bg=BG_COLOR)
    enabled_var = tk.BooleanVar()
    checkbutton = tk.Checkbutton(filter_frame, text=filter_text, variable=enabled_var, bg=BG_COLOR, fg=TEXT_COLOR,selectcolor=BG_COLOR, activeforeground=TEXT_COLOR)
    checkbutton.pack(side=tk.LEFT)
    applied_filters[filter_func] = {"enabled": enabled_var, "checkbutton": checkbutton, "frame": filter_frame}

    if filter_func not in [bw_image, flip_image, binary_image]:
        slider = tk.Scale(filter_frame, from_=0, to=10, orient=tk.HORIZONTAL, resolution=0.1, bg=BG_COLOR, fg=TEXT_COLOR, activebackground=ACTIVE_BTN_COLOR, highlightthickness=0, sliderrelief=tk.FLAT, sliderlength=20, troughcolor=BTN_COLOR)
        slider.pack(side=tk.RIGHT)

        if filter_func == resize_image:
            slider.set(1)
            slider.configure(from_=0.5, to=2)

        if filter_func == blur_image:
            slider.set(1)
            slider.configure(to=20)

        applied_filters[filter_func]["slider"] = slider

    filter_frame.pack(pady=5)

# تحويل الوضع الليلي/النهاري
def toggle_night_mode():
    global BG_COLOR, BTN_COLOR, TEXT_COLOR, ACTIVE_BTN_COLOR
    if BG_COLOR == "#A163F7":  # وضع النهار
        BG_COLOR = "#282a36"
        BTN_COLOR = "#44475a"
        TEXT_COLOR = "#f8f8f2"
        ACTIVE_BTN_COLOR = "#6272a4"
        night_mode_button.config(text="تشغيل الوضع النهاري")
    else:  # وضع الليل
        BG_COLOR = "#A163F7"
        BTN_COLOR = "#6F88FC"
        TEXT_COLOR = "#f8f8f2"
        ACTIVE_BTN_COLOR = "#45E3FF"
        night_mode_button.config(text="تشغيل الوضع الليلي")

    # تحديث الألوان
    root.configure(bg=BG_COLOR)
    logo_label.config(bg=BG_COLOR)
    load_button.config(bg=BTN_COLOR, activebackground=ACTIVE_BTN_COLOR, fg=TEXT_COLOR)
    apply_button.config(bg=BTN_COLOR, activebackground=ACTIVE_BTN_COLOR, fg=TEXT_COLOR)
    image_label_main.config(bg=BG_COLOR, fg=TEXT_COLOR)

    for filter_data in applied_filters.values():
        filter_data["checkbutton"].config(bg=BG_COLOR, selectcolor=BG_COLOR, fg=TEXT_COLOR)
        filter_data["frame"].config(bg=BG_COLOR)
        if "slider" in filter_data:
            filter_data["slider"].config(bg=BG_COLOR, activebackground=ACTIVE_BTN_COLOR, troughcolor=BTN_COLOR, fg=TEXT_COLOR)

# إضافة الزر لتبديل الوضع الليلي
night_mode_button = tk.Button(root, text="تشغيل الوضع الليلي", command=toggle_night_mode, bg=BTN_COLOR, fg=TEXT_COLOR, activebackground=ACTIVE_BTN_COLOR)
night_mode_button.pack(pady=10)

# تعبئة الأزرار والعناصر الرئيسية
load_button.pack(pady=10)
apply_button.pack(pady=10)
image_label_main.pack(pady=10)

# بدء البرنامج
root.mainloop()